In [1]:
import pandas as pd
import sqlite3
import plotly.express as px
import altair as alt

In [ ]:
conn = sqlite3.connect("db/lunch_db.sqlite3")

##### How many meals were available day before order day ay 18:00 o'clock?
- Some meals with same name has diffrent ID. Do not know reason of this.
- I did not find situation where ID have diffrent meal name. 
- I assume that every small changes in name, igredients or price required new ID for meal.

In [3]:
query = '''
select *
from dash_numbers_of_meals
'''
df = pd.read_sql(query, conn, parse_dates=['order_day'])

In [4]:
df.columns = ['order day', 'Meal ID', 'Meal name']
df.sample(4)

,order day,Meal ID,Meal name
21,2023-03-15,146,134
25,2023-03-21,139,113
28,2023-03-24,171,141
2,2023-02-16,65,65


In [5]:
df_melt = df.melt(
    id_vars='order day',
    var_name='counter_type',
    value_name='counter'
)
df_melt.sample(4)

,order day,counter_type,counter
50,2023-03-15,Meal name,134
5,2023-02-21,Meal ID,96
57,2023-03-24,Meal name,141
35,2023-02-22,Meal name,119


In [6]:
alt.Chart(df_melt).mark_line().encode(
    x=alt.X('order day', title='Order day'),
    y=alt.Y('counter', title='Unique number'),
    color='counter_type'
).properties(
    title="How many meals were available day before order day ay 18:00 o'clock?",
    width=600,
    height=200,
)

alt.Chart(...)

##### Restaurants rate ranking
- TOP3 Cochise Burgers, Sandwicz Shop - Kraków, Sandwicz Szop
- Bar Alf-BISTRO: big star, 4 place in overall rate and a lot of votes.
- Sandwicz Szop also not available anymore.

In [7]:
query = "select * from dash_restaurants_ranking"
df_rrr = pd.read_sql(query, conn, parse_dates=['rate_date'])
df_rrr.head()

,id_restaurants,name,rate_date,rate,rates_count
0,1710,Cochise Burgery,2023-07-04,4.56,219.0
1,1053,Sandwicz Szop - Kraków,2023-05-26,4.55,10551.0
2,1060,Sandwicz Szop,2023-05-25,4.54,30615.0
3,1847,Fit Morning- Kraków,2023-06-29,4.54,1725.0
4,437,Bar ALF - BISTRO,2023-07-04,4.53,49790.0


In [8]:
alt.Chart(df_rrr).mark_bar().encode(
    x=alt.X('name', title='Restaurant name').sort(alt.SortField('rate', order='descending')),
    y=alt.Y('rate', title='Rate'),
).properties(
    title="Restaurants rate ranking",
    width=600,
    height=200,
)


alt.Chart(...)

In [9]:
base = alt.Chart(df_rrr).encode(
    x=alt.X('name', title='Restaurant name').sort(alt.SortField('rate', order='descending')),
    y=alt.Y('rates_count', title='Number of rates'),
    text='rates_count',
).properties(
    title="Restaurants number of rates - sorted by overall rate",
    width=600,
    height=200,
)

base.mark_bar() + base.mark_text(dy=-10)

alt.LayerChart(...)

##### Restaurants rate ranking history
- Śniadaniowcy: replacement for Sandwich Szop receives a lot of rates vote in short time means they have a lot of orders.

In [10]:
query = '''
select * from dash_restaurants_ranking_history 
where id_restaurants in (437, 727, 1060, 1710, 438, 1836)

'''
df_rrrh = pd.read_sql(query, conn, parse_dates=['rate_date'])
df_rrrh.head()

,rate_date,id_restaurants,name,rate,rates_count
0,2023-02-10,437,Bar ALF - BISTRO,4.54,40080.0
1,2023-02-10,727,Catering Karp dla TME,4.34,14768.0
2,2023-02-10,438,Da Grasso Kossaka - Łódź,4.40,1840.0
3,2023-02-10,1060,Sandwicz Szop,4.54,22233.0
4,2023-02-11,437,Bar ALF - BISTRO,4.54,40163.0


In [21]:
alt.Chart(df_rrrh).mark_line().encode(
    x=alt.X('rate_date', title='Date'),
    y=alt.Y('rate', title='Rate', scale=alt.Scale(domain=[3.8, 4.7])),
    color=alt.Color('name', title='Restaurants name'),
).properties(
    title="Restaurants rate in time",
    width=600,
    height=200,
)

alt.Chart(...)

In [46]:
alt.Chart(df_rrrh).mark_line().encode(
    x='rate_date',
    y='rates_count',
    color='name',
).properties(
    title="Restaurants number of rates in time",
    width=600,
    height=200,
)

alt.Chart(...)

##### Average meal price
- it looks like prices are not in increaseing trend,
- peak in May, caused probably by holiday in Poland and only few restaurants offer meals.

In [61]:
query = '''
select * from dash_average_meal_price 
'''
df_amr = pd.read_sql(query, conn, parse_dates=['order_day'])
df_amr.head()

,order_day,avg_price,max_price,min_price
0,2023-03-27,16.399381,28,9
1,2023-03-28,18.875161,48,8
2,2023-03-29,18.690755,39,3
3,2023-03-30,16.430294,30,0
4,2023-03-31,19.490731,42,8


In [62]:
alt.Chart(df_amr).mark_line().encode(
    x=alt.X('order_day', title='Order day'),
    y=alt.Y('avg_price', title='Average meal price')
).properties(
    title="Average meal price",
    width=600,
    height=200,
)

alt.Chart(...)

In [63]:
alt.Chart(df_amr).mark_line().encode(
    x=alt.X('order_day', title='Order day'),
    y=alt.Y('max_price', title='Max meal price')
).properties(
    title="Max meal price",
    width=600,
    height=200,
)

alt.Chart(...)

In [64]:
alt.Chart(df_amr).mark_line().encode(
    x=alt.X('order_day', title='Order day'),
    y=alt.Y('min_price', title='Min meal price')
).properties(
    title="Minimal meal price",
    width=600,
    height=200,
)

alt.Chart(...)

##### Average restaurant meal price
- sandwich restaurant (like SandwichSzop, Sniadaniowcy, Fit-Morning) with much lower price than regular restaurants.
    

In [15]:
query = '''
select * 
from dash_average_restaurent_meal_price 
'''
df_armr = pd.read_sql(query, conn, parse_dates=['order_day'])
df_armr.head()

,restaurant_id,name,avg_meal_price
0,831,Tommy Burger,29.63
1,915,Hashtag Sushi II,29.33
2,493,Hashtag Sushi,26.33
3,1710,Cochise Burgery,25.52
4,663,KukuryQ,25.43


In [16]:
base = alt.Chart(df_armr).mark_bar().encode(
    x=alt.X('name', title='Restaurant name').sort(alt.SortField('avg_meal_price', order='descending')),
    y=alt.Y('avg_meal_price', title='Average meal price'),
    text = 'avg_meal_price'
).properties(
    title="Restaurants average price",
    width=600,
    height=200,
)

base.mark_bar() + base.mark_text(dy=-10)


alt.LayerChart(...)

##### Most expensive meal
- Fit Morning is only on restaurant which regular offers meal for meetings.
- Catering Karp dla TME - special meal for Easter.

In [17]:
query = '''
select * 
from dash_most_expensive_meal 
'''
df_mem = pd.read_sql(query, conn, parse_dates=['order_day'])
df_mem.head()

,mean_name_pl,max_price,name
0,Patera konferencyjna - Zestaw kanapek dla 5 -...,90,Fit Morning- Kraków
1,Big Boss Burger,56,Tommy Burger
2,"WYNOS WIELKANOCNY:Galantyna z kurczaka, rolada...",53,Catering Karp dla TME
3,Burger Totem,48,Cochise Burgery
4,SUSHI duży zestaw z rybą wędzoną,42,Sandwicz Szop - Gdynia


In [18]:
base = alt.Chart(df_mem).encode(
    x=alt.X('mean_name_pl', title='Meal').sort(alt.SortField('max_price', order='descending')),
    y=alt.Y('max_price', title='Price'),
    text = 'max_price'
).properties(
    title="Most expensive meal",
    width=600,
    height=200,
)

base.mark_bar() + base.mark_text(dy=-10)

alt.LayerChart(...)

##### Cheapest meal
- garlic sauce for pizza cost only 0.01 PLN. When sauce where for free people ordered it but did not eat.

In [19]:
query = '''
select * 
from dash_cheapest_meal 
'''
df_cm = pd.read_sql(query, conn, parse_dates=['order_day'])
df_cm.head()

,mean_name_pl,max_price,name
0,Sos czosnkowy,0.01,Da Grasso Kossaka - Łódź
1,Jabłko średnie - umyte,1.60,Fit Morning- Kraków
2,Banan,2.75,Fit Morning- Kraków
3,Bułka paryska,3.00,Catering Karp dla TME
4,Bułka poznańska,3.00,Catering Karp dla TME


In [20]:
base = alt.Chart(df_cm).encode(
    x=alt.X('mean_name_pl', title='Meal').sort(alt.SortField('max_price', order='ascending')),
    y=alt.Y('max_price', title='Price'),
    text = 'max_price'
).properties(
    title="Cheapest meal",
    width=600,
    height=200,
)

base.mark_bar() + base.mark_text(dy=-10)

alt.LayerChart(...)

##### Best Meals Ever
- 3 meals have 5.00 price rate with 22, 40 and 35 rates from customers.

In [65]:
query = '''
select * 
from dash_meals_best 
'''
df_dmb = pd.read_sql(query, conn, parse_dates=['order_day'])
df_dmb

,mean_name_pl,restaurant_name,max_rate,max_rates
0,Tostowa podwójna z pasztetem i warzywami,Catering Karp dla TME,5.000000,22
1,SAŁATKA Burak z kaszą pęczak i wędzonym twaroż...,Sandwicz Szop,5.000000,40
2,"Egzotyczna - owsianka z mango, brzoskwinią, wi...",Fit Morning-Łódzkie,5.000000,35
3,"ŚNIADANIE Burrito śniadaniowe z jajecznicą, sz...",Sandwicz Szop,4.973684,38
4,Smoothie Abracababra 250 ml,Fit Morning-Łódzkie,4.960000,114
5,DANIE OBIADOWE Butterchicken z ryżem basmati p...,Sandwicz Szop,4.944444,33
6,Ryż smażony z warzywami z surówką i sosem słod...,Ha Long,4.941176,36
7,JOGURT Tropical,Sandwicz Szop,4.941176,64
8,JOGURT Pudding ryżowy z wiśniami,Sandwicz Szop - Kraków,4.937500,40
9,KANAPKA Bajgiel włoski z jajkiem,Sandwicz Szop,4.933333,149


##### Worsts Meals Ever

In [67]:
query = '''
select * 
from dash_meals_worst 
'''
df_dmw = pd.read_sql(query, conn, parse_dates=['order_day'])
df_dmw

,mean_name_pl,restaurant_name,min_rate,max_rates
0,VegeMiso,Hashtag Sushi,3.070000,15
1,Sałatka quinoa,Foodstacja bistro&cafe,3.270000,15
2,NOWOŚĆ!! Burger Kukuryq,KukuryQ,3.062500,20
3,Oryginalne Skrzydełka Buffalo 6szt,Tommy Burger,3.526316,19
4,Zupa II: Meksykańska z zielonym groszkiem,Catering Karp dla TME,3.562500,16
5,"Danie dnia: Potrawka z kurczaka, ryż biały, ma...",Bar ALF - BISTRO,3.600000,15
6,"Wołowina po tajlandzku z ryżem białym, surówką...",Ha Long,3.222222,49
7,Wege: Spaghetti ala Bolognese - mniejsza porcja,Bar ALF - BISTRO,3.647059,17
8,ZUPA Ogórkowa,Sandwicz Szop,3.666667,18
9,DANIE OBIADOWE Orientalny udon z kurczakiem i ...,Sandwicz Szop - Gdynia,3.670000,15


##### Meals - bigest diffrence in rate

In [66]:
query = '''
select * 
from dash_meals_biggest_rate_diff
'''
df_mbdr = pd.read_sql(query, conn, parse_dates=['order_day'])
df_mbdr

,mean_name_pl,restaurant_name,max_rate,min_rate,rate_diff
0,JOGURT Malina,Sandwicz Szop - Gdynia,4.900000,3.330000,1.570000
1,Kanapka kurczak,Śniadaniowcy Łódź,4.360000,2.850000,1.510000
2,Smoothie zielone,Tommy Burger,4.800000,3.440000,1.360000
3,Sałatka z kurczakiem,Śniadaniowcy Łódź,4.641026,3.350000,1.291026
4,Sałatka z grillowanym kurczakiem,Śniadaniowcy Łódź,4.210000,2.940000,1.270000
5,"SAŁATKA DUŻA Ryż i tuńczyk - sałatka z ryżem, ...",Fit Morning-Łódzkie,4.710000,3.440000,1.270000
6,Fish burger,KukuryQ,4.571429,3.420000,1.151429
7,#Sushi D 350g + surówka,Hashtag Sushi II,4.571429,3.500000,1.071429
8,Zupa II: Borowikowa z makaronem i wołowiną,Catering Karp dla TME,4.562500,3.529412,1.033088
9,Kasza i łosoś - Sałatka z kuskus i wędzonym ło...,Fit Morning-Łódzkie,4.470000,3.440000,1.030000


In [69]:
query = '''
select * 
from dash_meals_biggest_rate_diff_top3
'''
df_mbrd3 = pd.read_sql(query, conn, parse_dates=['order_day'])

In [68]:
alt.Chart(df_mbrd3).mark_line().encode(
    x='order_day',
    y='rate',
    color='mean_name_pl',
).properties(
    title="Meals - biggest diffrence in rate",
    width=600,
    height=200,
)

alt.Chart(...)